In [1]:
import xml.etree.ElementTree as ET
import json

In [3]:
def convert(xml):

    tree = ET.parse(xml)
    root = tree.getroot()
    spacyJson = []
    ents = ["PRS", "ORG", "LOC", "TME"]
    
    for paragraph in root.iter('text'):
        paragraphDict = {}
        sentencesList = []
        
        sentencesDict = {}
        paragraphsList = []
        
        for sent in paragraph.iter('sentence'):
            sentsDict = {}    
            tokensDict = {}
            tokensList = []
            for child in sent:
                if child.tag == 'name':
                    for token in child.iter('w'):
                        nameDict = {'orth': token.text, 'tag': token.attrib['msd'].replace('.', '|'), "ner": "O"}
                        tokensList.append(nameDict)
                elif child.tag == 'ne' and child.attrib['type'] in ents:
                    entityTokenList = child.attrib['name'].split(' ')
                    for token in child.iter('w'):
                        neDict = {'orth': token.text, 'tag': token.attrib['msd'].replace('.', '|')}
                        if len(entityTokenList) == 1:
                            neDict['ner'] = 'U-' + child.attrib['type']
                            tokensList.append(neDict)
                            continue
                        elif token.text == entityTokenList[len(entityTokenList) - 1]:
                            neDict['ner'] = 'L-' + child.attrib['type']
                            tokensList.append(neDict)
                            continue
                        elif token.text == entityTokenList[0]:
                            neDict['ner'] = 'B-' + child.attrib['type']
                            tokensList.append(neDict)
                            continue
                        else:
                            neDict['ner'] = 'I-' + child.attrib['type']
                            tokensList.append(neDict)
                elif child.tag == 'ne':
                    for token in child.iter('w'):
                        tokenDict = {'orth': token.text, 'tag': token.attrib['msd'].replace('.', '|'), "ner": "O"}
                        tokensList.append(tokenDict)
                elif child.tag == 'w':
                    tokenDict = {'orth': child.text, 'tag': child.attrib['msd'].replace('.', '|'), "ner": "O"}
                    tokensList.append(tokenDict)
                else:
                    continue
            sentsDict['tokens'] = tokensList
            sentencesList.append(sentsDict)
        sentencesDict['sentences'] = sentencesList
        paragraphsList.append(sentencesDict)
        paragraphDict['paragraphs'] = paragraphsList
        spacyJson.append(paragraphDict)
    
    return spacyJson

In [4]:
def printToFile(file, content):
    with open(file, 'w', encoding='utf-8') as f:
        json.dump(content, f, ensure_ascii=False, indent=2)
    

In [5]:
spacyTrain = convert('../Data/Datasets/Common/suc3train.xml')

printToFile('../Data/Datasets/ner/suc30train.json', spacyTrain)
